In [1]:
from binance.client import Client
from config import CRYPTO_TABLE_NAME, COINS
from creds import BINANCE_API_KEY, BINANCE_SECRET_KEY, HOST, DB, USER, PW, SCHEMA
from db_connector import engine
import pandas as pd
import datetime

In [2]:
client = Client(BINANCE_API_KEY, BINANCE_SECRET_KEY)

In [3]:
def get_historical_data_from_to(symbol, from_date, to_date):
    # Execute the query from binance - timestamps must be converted to strings !
    candles = client.get_historical_klines(
        symbol, Client.KLINE_INTERVAL_1MINUTE, str(from_date), str(to_date))
    # Create a dataframe to label all the columns returned by binance so we work with them later.
    df = pd.DataFrame(candles, columns=['starttime', 'open', 'high', 'low', 'close', 'basevolume',
                      'stoptime', 'quotevolume', 'numberOfTrades', 'takerBuyBaseVol', 'takerBuyQuoteVol', 'ignore'])
    df['symbol'] = symbol
    df = df.loc[:, ['starttime', 'stoptime', 'symbol', 'open',
                    'close', 'high', 'low', 'basevolume', 'quotevolume']]
    df["market_cap"] = 0
    # Data cleaning
    df.open = df.open.astype(float)
    df.close = df.close.astype(float)
    df.high = df.high.astype(float)
    df.low = df.low.astype(float)
    df.market_cap = df.market_cap.astype(float)
    df.starttime = pd.to_datetime(df.starttime, unit='ms')
    df.stoptime = pd.to_datetime(df.stoptime, unit='ms')
    return df

In [9]:
df_exist = pd.read_sql(f"select * from \"{CRYPTO_TABLE_NAME}\"", engine.connect())
df_exist

,starttime,symbol,open,close,high,low,basevolume,quotevolume,market_cap
0,2021-11-04 23:44:00,BNBEUR,484.00000,484.10000,484.20000,483.90000,7.09400000,3433.97340000,8.064374e+10
1,2021-11-04 23:44:00,UNIEUR,22.29000,22.29000,22.29000,22.29000,2.47000000,55.05630000,1.399553e+10
2,2021-11-04 23:44:00,TRXEUR,0.08943,0.08942,0.08943,0.08941,10354.00000000,925.86166000,6.397968e+09
3,2021-11-04 23:45:00,BNBEUR,484.20000,484.00000,484.20000,483.90000,4.52500000,2190.51940000,8.062708e+10
4,2021-11-04 23:45:00,BTCEUR,53116.19000,53119.11000,53130.00000,53110.88000,0.27626000,14673.71541220,1.001170e+12
...,...,...,...,...,...,...,...,...,...
1210786,2021-11-04 23:44:00,SOLEUR,214.34000,214.76000,214.92000,214.21000,20.05000000,4303.23280000,6.445685e+10
1210787,2021-11-04 23:44:00,MATICEUR,1.66270,1.66270,1.66270,1.66270,0.00000000,0.00000000,1.130982e+10
1210788,2021-11-04 23:44:00,BTCEUR,53097.12000,53123.26000,53131.63000,53087.02000,0.37132000,19719.44095510,1.001249e+12
1210789,2021-11-04 23:44:00,DOGEEUR,0.22780,0.22760,0.22780,0.22760,946.00000000,215.39720000,2.998463e+10


In [11]:
df_exist = df_exist.loc[df_exist.starttime == df_exist.starttime.max(), ["starttime", "symbol", "close", "market_cap"]]

In [14]:
df_exist

,starttime,symbol,close,market_cap
706962,2021-12-24 04:24:00,DOGEEUR,0.16090,2.134713e+10
706963,2021-12-24 04:24:00,BTCEUR,45049.05000,8.524100e+11
706964,2021-12-24 04:24:00,XRPEUR,0.85740,4.077559e+10
706965,2021-12-24 04:24:00,MATICEUR,2.28870,1.624541e+10
706966,2021-12-24 04:24:00,TRXEUR,0.07287,7.431488e+09
706984,2021-12-24 04:24:00,ETHEUR,3614.39000,4.298670e+11
706985,2021-12-24 04:24:00,BNBEUR,480.50000,8.026345e+10
706986,2021-12-24 04:24:00,ADAEUR,1.28200,4.295925e+10
706987,2021-12-24 04:24:00,UNIEUR,15.53000,9.750545e+09
706988,2021-12-24 04:24:00,SOLEUR,166.09000,5.130242e+10


In [12]:
def add_market_cap(df, symbol):
    # sort that first value in db (time wise) is first value in df
    df = df.sort_values(by="starttime", ascending=False)
    # calculate pct_change of close price
    df["pct_change"] = df["close"].pct_change(periods=1)
    df["pct_change"] += 1
    df.iloc[0, 10] = 1
    # for easier interpolation pre calculate cumulative product of pct_change
    df["base_change"] = df.loc[:,'pct_change'].cumprod()
    # get market cap to interpoilate from
    market_cap = df_exist.loc[df_exist.symbol == symbol, "market_cap"].values[0]
    # interpolate market cap
    df.market_cap = market_cap * df.base_change
    # drop not needed columns and rows
    df = df.drop(["pct_change", "base_change"], axis=1)
    df = df.iloc[1:]

    return df

In [13]:
def get_historical_data_all(to_date, from_date): 
    first = True
    for coin in COINS:
        symbol = f"{coin}EUR"
        to_date = to_date
        from_date = from_date
        df = get_historical_data_from_to(symbol, from_date, to_date)
        df = add_market_cap(df, symbol)
        if first:
            df_all = df
            first = False
        else:
            df_all = df_all.append(df)
    
    return df_all.sort_values(by="starttime", ascending=False)

In [15]:
to_date = pd.to_datetime("2022-01-07 15:55:00")
from_date = pd.to_datetime("2021-12-24 04:25:00")
print(to_date)
print(from_date)

2022-01-07 15:55:00
2021-12-24 04:25:00


In [16]:
df = get_historical_data_all(to_date, from_date)
df

,starttime,stoptime,symbol,open,close,high,low,basevolume,quotevolume,market_cap
20795,2022-01-07 15:00:00,2022-01-07 15:00:59.999,TRXEUR,0.06016,0.06016,0.06016,0.06016,850.00000000,51.13600000,7.436433e+09
20794,2022-01-07 14:59:00,2022-01-07 14:59:59.999,MATICEUR,1.90490,1.90100,1.90490,1.90100,1559.20000000,2967.95335000,1.627623e+10
20794,2022-01-07 14:59:00,2022-01-07 14:59:59.999,UNIEUR,14.09000,14.09000,14.09000,14.09000,0.00000000,0.00000000,9.785269e+09
20794,2022-01-07 14:59:00,2022-01-07 14:59:59.999,TRXEUR,0.06013,0.06019,0.06019,0.06012,31795.00000000,1911.64829000,7.440141e+09
20793,2022-01-07 14:58:00,2022-01-07 14:58:59.999,SOLEUR,123.92000,123.76000,123.92000,123.60000,9.07000000,1122.36310000,5.133561e+10
...,...,...,...,...,...,...,...,...,...,...
0,2021-12-24 04:25:00,2021-12-24 04:25:59.999,SOLEUR,166.09000,166.09000,166.09000,166.09000,0.00000000,0.00000000,6.889407e+10
0,2021-12-24 04:25:00,2021-12-24 04:25:59.999,ETHEUR,3614.96000,3615.33000,3615.33000,3614.96000,0.01900000,68.68683000,5.471317e+11
0,2021-12-24 04:25:00,2021-12-24 04:25:59.999,XRPEUR,0.85820,0.85840,0.85910,0.85820,2221.00000000,1906.60030000,5.223365e+10
0,2021-12-24 04:25:00,2021-12-24 04:25:59.999,BNBEUR,480.50000,480.70000,480.70000,480.50000,1.05100000,505.09670000,9.699005e+10


In [17]:
df.groupby(by="symbol").agg({
    "starttime": ["min", "max", "count"]
})

starttime                           
                         min                 max  count
symbol                                                 
ADAEUR   2021-12-24 04:25:00 2022-01-07 14:56:00  20792
BNBEUR   2021-12-24 04:25:00 2022-01-07 14:57:00  20793
BTCEUR   2021-12-24 04:25:00 2022-01-07 14:56:00  20792
DOGEEUR  2021-12-24 04:25:00 2022-01-07 14:58:00  20794
ETHEUR   2021-12-24 04:25:00 2022-01-07 14:56:00  20792
MATICEUR 2021-12-24 04:25:00 2022-01-07 14:59:00  20795
SOLEUR   2021-12-24 04:25:00 2022-01-07 14:58:00  20794
TRXEUR   2021-12-24 04:25:00 2022-01-07 15:00:00  20796
UNIEUR   2021-12-24 04:25:00 2022-01-07 14:59:00  20795
XRPEUR   2021-12-24 04:25:00 2022-01-07 14:57:00  20793

In [18]:
df = df.loc[: ,["starttime", "symbol","open","close","high","low","basevolume","quotevolume","market_cap"]]
df.head(12)

,starttime,symbol,open,close,high,low,basevolume,quotevolume,market_cap
20795,2022-01-07 15:00:00,TRXEUR,0.06016,0.06016,0.06016,0.06016,850.00000000,51.13600000,7.436433e+09
20794,2022-01-07 14:59:00,MATICEUR,1.90490,1.90100,1.90490,1.90100,1559.20000000,2967.95335000,1.627623e+10
20794,2022-01-07 14:59:00,UNIEUR,14.09000,14.09000,14.09000,14.09000,0.00000000,0.00000000,9.785269e+09
20794,2022-01-07 14:59:00,TRXEUR,0.06013,0.06019,0.06019,0.06012,31795.00000000,1911.64829000,7.440141e+09
20793,2022-01-07 14:58:00,SOLEUR,123.92000,123.76000,123.92000,123.60000,9.07000000,1122.36310000,5.133561e+10
20793,2022-01-07 14:58:00,TRXEUR,0.06030,0.06030,0.06030,0.06030,0.00000000,0.00000000,7.453738e+09
20793,2022-01-07 14:58:00,DOGEEUR,0.13630,0.13630,0.13630,0.13630,253.00000000,34.48390000,2.142573e+10
20793,2022-01-07 14:58:00,MATICEUR,1.90640,1.90310,1.90640,1.90200,158.20000000,301.31736000,1.629421e+10
20793,2022-01-07 14:58:00,UNIEUR,14.09000,14.09000,14.09000,14.09000,0.00000000,0.00000000,9.785269e+09
20792,2022-01-07 14:57:00,MATICEUR,1.90890,1.91160,1.91160,1.90890,68.10000000,130.11486000,1.636699e+10


In [19]:
df.groupby(by="symbol").agg({
    "starttime": ["min", "max", "count"],
    "market_cap": "last",
    "close": "last"
})

starttime                               market_cap  \
                         min                 max  count          last   
symbol                                                                  
ADAEUR   2021-12-24 04:25:00 2022-01-07 14:56:00  20792  5.063337e+10   
BNBEUR   2021-12-24 04:25:00 2022-01-07 14:57:00  20793  9.699005e+10   
BTCEUR   2021-12-24 04:25:00 2022-01-07 14:56:00  20792  1.035481e+12   
DOGEEUR  2021-12-24 04:25:00 2022-01-07 14:58:00  20794  2.527701e+10   
ETHEUR   2021-12-24 04:25:00 2022-01-07 14:56:00  20792  5.471317e+11   
MATICEUR 2021-12-24 04:25:00 2022-01-07 14:59:00  20795  1.960425e+10   
SOLEUR   2021-12-24 04:25:00 2022-01-07 14:58:00  20794  6.889407e+10   
TRXEUR   2021-12-24 04:25:00 2022-01-07 15:00:00  20796  9.005055e+09   
UNIEUR   2021-12-24 04:25:00 2022-01-07 14:59:00  20795  1.078533e+10   
XRPEUR   2021-12-24 04:25:00 2022-01-07 14:57:00  20793  5.223365e+10   

                close  
                 last  
symbol                 
ADAEUR        1.28000  
BNBEUR      480.70000  
BTCEUR    45050.07000  
DOGEEUR       0.16080  
ETHEUR     3615.33000  
MATICEUR      2.28970  
SOLEUR      166.09000  
TRXEUR        0.07285  
UNIEUR       15.53000  
XRPEUR        0.85840

In [20]:
df.to_sql(CRYPTO_TABLE_NAME, engine, if_exists='append', index=False)

In [31]:
df1 = df.copy()

In [42]:
df1["mc_change"] = df1.loc[df1.symbol == "BTCEUR", :].market_cap.pct_change()
df1["cl_change"] = df1.loc[df1.symbol == "BTCEUR", "close"].pct_change()
mc_change = df1.loc[df1.symbol == "BTCEUR", :].mc_change
cl_change = df1.loc[df1.symbol == "BTCEUR", :].cl_change


In [48]:
df1.loc[df1.symbol == "BTCEUR", ["cl_change", "mc_change"]].describe()

,cl_change,mc_change
count,3.118000e+04,3.118000e+04
mean,-1.006158e-05,-1.006158e-05
std,8.941820e-04,8.941820e-04
min,-2.277254e-02,-2.277254e-02
25%,-4.275290e-04,-4.275290e-04
50%,1.888915e-07,1.888915e-07
75%,4.166054e-04,4.166054e-04
max,2.241690e-02,2.241690e-02


In [50]:
ser = mc_change - cl_change
ser.max()

2.220446049250313e-16

In [9]:
# df.to_sql(CRYPTO_TABLE_NAME, engine, if_exists='append', index=False)